# Cleaning txt file

In [ ]:
with open ('drseuss.txt', 'r+', encoding="utf-8", errors='ignore') as f:
     with open ('drseuss_new.txt', 'w+') as nf:
        lines = f.readlines()
        replacements = [('\n', ''), ('   ', ' '), ('  ', ' '), ('\"', ''), 
                        ('...', '.'), ('....', '.'), ('?', '.'), ('!', '.')]
        lines = ' '.join(lines)
        for current, new in replacements:
            lines = lines.replace(current,new)
        lines = lines.lower()
        nf.write(lines)

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

/anaconda3/envs/PythonData/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Training Model

In [2]:
filename = "data/combinedText.txt"
raw_text = open(filename).read()


In [3]:
list(set(raw_text))[0]

'k'

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  55677
Total Vocab:  48


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  55577


In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list, verbose=1)

# Running Model with Seed Input

In [ ]:
filename = "weights-improvement-20-1.7518.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')



int_to_char = dict((i, c) for i, c in enumerate(chars))

pattern = "the cat in the hat said to the bat that i’m fat. oh no, said the bat, i’m not fat. what say you bird"

x_train = [char_to_int[i] for i in pattern]
x_train = numpy.reshape(x_train, (1,len(x_train),1))
x_train = x_train / float(n_vocab)

for i in range(50):
    x_train = numpy.reshape(x_train, (1,-1,1))
    prediction = model.predict(x_train, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
#     seq_in = [int_to_char[value] for value in x_train]

    sys.stdout.write(result)
    x_train = numpy.append(x_train, index)
    x_train = x_train[1:len(x_train)]
    
    

In [ ]:
temp = model.predict(numpy.reshape(x_train, (1,-1,1)))

In [ ]:
print(list(zip(temp[0], int_to_char.values())))

In [ ]:
int_to_char[15]
int_to_char[26]